In [2]:
import torch
from prettytable import PrettyTable

In [16]:
if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"
print("Selected device is",DEVICE)

input_features_length = 15

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

Selected device is cuda


In [18]:
%run ../shared_functions.py
%run ../my_shared_functions.py

<Figure size 640x480 with 0 Axes>

### MLP

In [29]:
class FraudMLPHypertuned(torch.nn.Module):
    def __init__(self, input_size,hidden_size=500,num_layers=2,p=0.2):
        super(FraudMLPHypertuned, self).__init__()
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.p = p
        
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        
        self.fc_hidden=[]
        for i in range(num_layers-1):
            self.fc_hidden.append(torch.nn.Linear(self.hidden_size, self.hidden_size).to(DEVICE))
            self.fc_hidden.append(torch.nn.ReLU())
            
        self.fc2 = torch.nn.Linear(self.hidden_size, 1)
        self.sigmoid = torch.nn.Sigmoid()
        
        self.dropout = torch.nn.Dropout(self.p)
        
    def forward(self, x):
        
        hidden = self.fc1(x)
        hidden = self.relu(hidden)             
        hidden = self.dropout(hidden)
        
        for layer in self.fc_hidden:
            hidden=layer(hidden)
            hidden = self.dropout(hidden)
        
        output = self.fc2(hidden)
        output = self.sigmoid(output)
        
        return output
    
mlp_model = FraudMLPHypertuned(input_size=input_features_length)
_ = count_parameters(mlp_model)

+------------+------------+
|  Modules   | Parameters |
+------------+------------+
| fc1.weight |    7500    |
|  fc1.bias  |    500     |
| fc2.weight |    500     |
|  fc2.bias  |     1      |
+------------+------------+
Total Trainable Params: 8501


### Autoencoder

In [26]:
autoencoder_model = SimpleAutoencoder(input_features_length, 100, 20)
autoencoder_parameters = count_parameters(autoencoder_model)

+------------+------------+
|  Modules   | Parameters |
+------------+------------+
| fc1.weight |    1500    |
|  fc1.bias  |    100     |
| fc2.weight |    2000    |
|  fc2.bias  |     20     |
| fc3.weight |    2000    |
|  fc3.bias  |    100     |
| fc4.weight |    1500    |
|  fc4.bias  |     15     |
+------------+------------+
Total Trainable Params: 7235


### Autoencoder + MLP

In [28]:
autoencoder_mlp_model = SimpleFraudMLPWithDropout(input_features_length + 1, 100, 0.2)
autoencoder_mlp_parameters = count_parameters(autoencoder_mlp_model)
print(autoencoder_parameters + autoencoder_mlp_parameters)

+------------+------------+
|  Modules   | Parameters |
+------------+------------+
| fc1.weight |    1600    |
|  fc1.bias  |    100     |
| fc2.weight |    100     |
|  fc2.bias  |     1      |
+------------+------------+
Total Trainable Params: 1801
9036


### CNN

In [30]:
cnn_model = FraudConvNetWithDropout(input_features_length, hidden_size=500, conv2_params=(100,2), p=0.2)
_ = count_parameters(cnn_model)

+--------------+------------+
|   Modules    | Parameters |
+--------------+------------+
| conv1.weight |    3000    |
|  conv1.bias  |    100     |
| conv2.weight |   20000    |
|  conv2.bias  |    100     |
|  fc1.weight  |   50000    |
|   fc1.bias   |    500     |
|  fc2.weight  |    500     |
|   fc2.bias   |     1      |
+--------------+------------+
Total Trainable Params: 74201


### LSTM

In [31]:
lstm_model = FraudLSTM(input_features_length, hidden_size=500, dropout_lstm=0.2)
_ = count_parameters(lstm_model)

+-------------------+------------+
|      Modules      | Parameters |
+-------------------+------------+
| lstm.weight_ih_l0 |    6000    |
| lstm.weight_hh_l0 |   40000    |
|  lstm.bias_ih_l0  |    400     |
|  lstm.bias_hh_l0  |    400     |
|     fc1.weight    |   50000    |
|      fc1.bias     |    500     |
|     fc2.weight    |    500     |
|      fc2.bias     |     1      |
+-------------------+------------+
Total Trainable Params: 97801


### LSTM Attention

In [32]:
lstm_attention_model = FraudLSTMWithAttention(input_features_length, hidden_size = 500, dropout_lstm=0.2)
_ = count_parameters(lstm_attention_model)

+-----------------------------+------------+
|           Modules           | Parameters |
+-----------------------------+------------+
|      lstm.weight_ih_l0      |    6000    |
|      lstm.weight_hh_l0      |   40000    |
|       lstm.bias_ih_l0       |    400     |
|       lstm.bias_hh_l0       |    400     |
|          ff.weight          |    1500    |
|           ff.bias           |    100     |
| attention.linear_out.weight |   20000    |
|  attention.linear_out.bias  |    100     |
|          fc1.weight         |   50000    |
|           fc1.bias          |    500     |
|          fc2.weight         |    500     |
|           fc2.bias          |     1      |
+-----------------------------+------------+
Total Trainable Params: 119501
